# Random Forests Implementation from Scratch 
- Try Analysing what is happening 
- Don't get stuck for long


# Sample Random Forest Implementation

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Load iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Initialize random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100)

# Fit the model to the data
rf.fit(X, y)

# Make predictions on new data
X_test = [[5.0, 3.5, 1.3, 0.2], [6.0, 3.0, 4.8, 1.8], [7.3, 2.9, 6.3, 1.8]]
y_pred = rf.predict(X_test)

print(y_pred)

[0 2 2]


# Decision Tree Implementation 

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris

# Load iris dataset
iris = load_iris()
X = iris.data
print(X.shape)
y = iris.target

# Initialize decision tree classifier with max_depth=3
dt = DecisionTreeClassifier(max_depth=3)

# Fit the model to the data
dt.fit(X, y)

# Make predictions on new data
X_test = [[5.0, 3.5, 1.3, 0.2], [6.0, 3.0, 4.8, 1.8], [7.3, 2.9, 6.3, 1.8]]
y_pred = dt.predict(X_test)

print(y_pred)

(150, 4)
(150,)
[0 2 2]
